In [ ]:
import sys
sys.path.append('/Users/paolamartire/shocks')

import Utilities.prelude
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import k3match
from scipy.optimize import fsolve
import pickle 

from Utilities.operators import make_tree, calc_grad

# %matplotlib widget # (to go back normal %matplotlib inline)
# mpl.rcParams['figure.dpi'] = 100

In [ ]:
delta = 0.039
z_chosen = 0 
save = False

In [ ]:
# def ray_tracer(sim_tree, X, Y, Z, x_zone, y_zone, z_zone, idx, div, dir, delta, direction):
#     point = np.array([x_zone[idx], y_zone[idx], z_zone[idx]])
#     div_v = div[idx]
#     dir_sh = np.array(dir[idx])

#     check_zone = 0 # to check if the new point is in the shock zone
#     while check_zone < delta:
#         # move along shock direction 
#         if direction == 'post':
#             new_point = point - dir_sh * delta
#         elif direction == 'pre':
#             new_point = point + dir_sh * delta
        
#         # find the nearest points in the grid and in the shock zone
#         _, i_tree  = sim_tree.query(new_point)
#         x_new = X[i_tree]
#         y_new = Y[i_tree]
#         z_new = Z[i_tree]
#         point = [x_new, y_new, z_new]

#         (idx_zone, dist) = k3match.nearest_cartesian(x_zone, y_zone, z_zone, x_new, y_new, z_new)
#         idx_zone = int(idx_zone)
#         check_zone = dist 
                    
#         div_next = div[idx_zone]
#         dir_next = dir[idx_zone]

#         # if lower div v, you discard the ray.
#         if div_next < div_v:
#             # print('pass')
#             return False # so then you don't take this cell

#         # if opposite direction in shocks, you turn/stop.
#         if np.dot(dir_sh, dir_next) < 0:
#             check_zone = 1e4 # so you exit from the while
 
#     final_tree_index = i_tree
    
#     return final_tree_index

In [ ]:
def shock_direction(sim_tree, X, Y, Z, Temp, point, delta):
    """ Find shock direction according eq.(5) by Schaal14 in the point of coordinates indices idx.
    Parameters
    -----------
    sim_tree: nDarray.
            Tree where to search the point.
    X, Y, Z, Temp: arrays.
            Coordinates and temperature of the points of the tree.
    point: array.
            Starting point.
    delta: float.
            Step between 2 neighbours.
    Returns
    -----------
    ds: array.
        Shock direction (vector of 3 components).
    """
    grad = calc_grad(sim_tree, X, Y, Z, Temp, point, delta)
    magnitude = np.linalg.norm(grad)
    
    if np.logical_and(np.abs(grad[0])<0.1, np.logical_and(np.abs(grad[1])<0.1, np.abs(grad[2])<0.1)):
        ds = [0, 0, 0]
    else:
        ds = - np.divide(grad,magnitude)
    return ds

def ray_tracer(sim_tree, X, Y, Z, are_u_shock, x_zone, y_zone, z_zone, idx, div, dir, delta, direction):
    point = np.array([x_zone[idx], y_zone[idx], z_zone[idx]])
    div_v = div[idx]
    dir_sh = np.array(dir[idx])

    if direction == 'post':
        delta = - delta

    check_zone = True # to check if the new point is in the shock zone
    while check_zone == True:
        new_point = point + dir_sh * delta
        # move along shock direction 
        
        # find the nearest points in the grid and in the shock zone
        _, i_tree  = sim_tree.query(new_point)
        check_zone = are_u_shock[i_tree]

        if check_zone == True:
            # dumb way 
            x_new = X[i_tree]
            y_new = Y[i_tree]
            z_new = Z[i_tree]
            point = [x_new, y_new, z_new]
            idx_zone, _ = k3match.nearest_cartesian(x_zone, y_zone, z_zone, x_new, y_new, z_new)
            idx_zone = int(idx_zone)
            div_next = div[idx_zone]
            dir_next = dir[idx_zone]

            # if lower div v, you discard the ray.
            if div_next < div_v:
                # print('pass')
                return False # so then you don't take this cell

            # if opposite direction in shocks, you turn/stop.
            if np.dot(dir_sh, dir_next) < 0:
                check_zone = False # so you exit from the while
 
    final_tree_index = i_tree
    
    return final_tree_index

def shock_surface(sim_tree, X, Y, Z, Temp, are_u_shock, x_zone, y_zone, z_zone, div, dir, delta):
    surface_mach = []
    surface_x = []
    surface_y = []
    surface_z = []
    post_x = []
    post_y = []
    post_z = []
    pre_x = []
    pre_y = []
    pre_z = []

    # loop over all the cells in the shock zone
    for idx in range(len(x_zone)):
        print(idx)
        post_tree_index = ray_tracer(sim_tree, X, Y, Z, are_u_shock, x_zone, y_zone, z_zone, idx, div, dir, delta, direction = 'post')
        if post_tree_index == False:
            continue
        else:
            Tpost = Temp[post_tree_index]
            pre_tree_index = ray_tracer(sim_tree, X, Y, Z, are_u_shock, x_zone, y_zone, z_zone, idx, div, dir, delta, direction = 'pre')
            if pre_tree_index == False:
                continue
            else:
                Tpre = Temp[pre_tree_index]

                bump = Tpost/Tpre
                # check if the bump is in the same direction of ds. CHECK THIS
                if bump < 1:
                    continue 

                surface_x.append(x_zone[idx])
                surface_y.append(y_zone[idx])
                surface_z.append(z_zone[idx])
                post_x.append(X[post_tree_index])
                post_y.append(Y[post_tree_index])
                post_z.append(Z[post_tree_index])
                pre_x.append(X[pre_tree_index])
                pre_y.append(Y[pre_tree_index])
                pre_z.append(Z[pre_tree_index])

                def temperature_bump(mach):
                    gamma = 5/3
                    to_solve =  ((gamma-1) * mach**2 + 2) * (2 * gamma * mach**2 - (gamma-1)) / ((gamma+1)**2 * mach**2) - bump
                    to_solve = float(to_solve)
                    return to_solve
                
                mach = fsolve(temperature_bump, 1.5)
                surface_mach.append(mach)
            
    surface_mach = np.array(surface_mach)
    surface_x = np.array(surface_x)
    surface_y = np.array(surface_y)
    surface_z = np.array(surface_z)
    post_x = np.array(post_x)
    post_y = np.array(post_y)
    post_z = np.array(post_z)
    pre_x = np.array(pre_x)
    pre_y = np.array(pre_y)
    pre_z = np.array(pre_z)

    return surface_mach, surface_x, surface_y, surface_z, post_x, post_y, post_z,  pre_x, pre_y, pre_z


In [ ]:
# Import data
shockzone = np.loadtxt(f'OLDcode/ArbitraryDist/shockzone.txt')
shockdir = np.loadtxt(f'OLDcode/ArbitraryDist/shockdir.txt')

x_zone = np.array(shockzone[0])
y_zone = shockzone[1]
z_zone = shockzone[2]
div = shockzone[3]
Tzone = shockzone[4]

x_dir = shockdir[0]
y_dir = shockdir[1]
z_dir = shockdir[2]
dir = []
for i in range(len(x_dir)):
    dir.append([x_dir[i], y_dir[i], z_dir[i]])

In [ ]:
sim_tree, X, Y, Z, Vol, VX, VY, VZ, Den, Press, Temp = make_tree('data_sim')

In [ ]:
with open('OLDcode/ArbitraryDist/shockbool.pkl', 'rb') as filebool:
    are_u_shock = pickle.load(filebool)

In [ ]:
surface_mach, surface_x, surface_y, surface_z, post_x, post_y, post_z, pre_x, pre_y, pre_z = shock_surface(sim_tree, X, Y, Z, Temp, are_u_shock, x_zone, y_zone, z_zone, div, dir, delta)

In [ ]:
if save == True:
    surface_mach_txt = np.concatenate(surface_mach)
    with open(f'OLDcode/ArbitraryDist/shocksurface.txt', 'w') as file:
        file.write(f'# Coordinates of the points in the shock surface \n# X \n') 
        file.write(' '.join(map(str, surface_x)) + '\n')
        file.write('# Y \n') 
        file.write(' '.join(map(str, surface_y)) + '\n')
        file.write('# Z \n') 
        file.write(' '.join(map(str, surface_z)) + '\n')
        file.write('# mach number v \n') 
        file.write(' '.join(map(str, surface_mach_txt)) + '\n')
        file.close()

In [ ]:
len(surface_mach)

In [ ]:
outlier = np.argmin(surface_mach[np.logical_and(surface_z<z_chosen+delta, surface_z>z_chosen-delta)])

In [ ]:
plt.plot(surface_mach[np.logical_and(surface_z<z_chosen+delta, surface_z>z_chosen-delta)], c = 'k')
# plt.scatter(outlier, surface_mach[np.logical_and(surface_z<z_chosen+delta, surface_z>z_chosen-delta)][outlier], c = 'r')
plt.ylabel(r'$\mathcal{M}$', fontsize = 15)
plt.title(f'Mach nuber, z=0, fixed step = {delta}', fontsize = 18)
plt.savefig(f'OLDcode/ArbitraryDist/ARBDISTmach_z{z_chosen}.png')
plt.show()

In [ ]:
# Cross section
slice_zone = np.logical_and(z_zone<z_chosen+delta, z_zone>z_chosen-delta)
zone_cross_x = x_zone[slice_zone]
zone_cross_y = y_zone[slice_zone]

slice_surface = np.logical_and(surface_z<z_chosen+delta, surface_z>z_chosen-delta)
surface_cross_x = surface_x[slice_surface]
surface_cross_y = surface_y[slice_surface]

slice_post = np.logical_and(post_z<z_chosen+delta, post_z>z_chosen-delta)
zone_post_x = post_x[np.logical_and(slice_surface, slice_post)]
zone_post_y = post_y[np.logical_and(slice_surface, slice_post)]

slice_pre = np.logical_and(pre_z<z_chosen+delta, pre_z>z_chosen-delta)
zone_pre_x = pre_x[np.logical_and(slice_surface, slice_pre)]
zone_pre_y = pre_y[np.logical_and(slice_surface, slice_pre)]


In [ ]:
# Maybe for every cell of the shock surface you don't see pre/post because they are not on the same layers
fig, ax = plt.subplots(1,1, figsize = (7,7))
ax.plot(zone_cross_x, zone_cross_y, 'ks', markerfacecolor='none', ms = 5, markeredgecolor='coral', label = 'Shock zone')
ax.plot(surface_cross_x, surface_cross_y, 'ks', markerfacecolor='maroon', ms = 5, markeredgecolor='maroon',  alpha = 0.8, label = 'Shock surface')


ax.plot(zone_post_x, zone_post_y, 'ks', markerfacecolor='sandybrown', ms = 5, markeredgecolor='none', alpha = 0.8,  label = 'Post shock')
ax.plot(zone_pre_x, zone_pre_y, 'ks', markerfacecolor='lightskyblue', ms = 5, markeredgecolor='none', alpha = 0.8,  label = 'Pre shock')

# ax.plot(surface_cross_x[outlier], surface_cross_y[outlier], 'ks', markerfacecolor='r', ms = 7, markeredgecolor='r',  label = 'outlier')
# ax.plot(post_x[slice_surface][outlier], post_y[slice_surface][outlier], 'ks', markerfacecolor='orange', ms = 7, markeredgecolor='orange',  label = 'outlier post')
# ax.plot(pre_x[slice_surface][outlier], pre_y[slice_surface][outlier], 'ks', markerfacecolor='b', ms = 7, markeredgecolor='b',  label = 'outlier pre')

ax.set_xlim(-1.1,1.1)
ax.set_ylim(-1.1,1.1)
ax.set_xlabel('X', fontsize = 18)
ax.set_ylabel('Y', fontsize = 18)
ax.legend()
ax.set_title(f'z = {z_chosen}, fixed step = {delta}', fontsize = 16)
plt.grid()
plt.savefig(f'OLDcode/ArbitraryDist/ARBDISTshocksurface_z{z_chosen}.png')